In [1]:
# from kaggle.api.kaggle_api_extended import KaggleApi
# import kagglehub
# import kaggle
# Initiliaze API

In [2]:

from sklearn import *
import pandas as pd
import seaborn as sns
# Donwload Dataset via API unzipped
# api.dataset_download_files(dataset="prathamtripathi/drug-classification", unzip=True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [18]:
lol_df = pd.read_csv("Challenger_Ranked_Games_15minute.csv")

In [4]:
# Count the number of differences
difference_count = (lol_df['blueWins'] != lol_df['redWins']).sum()

print(f"Number of differences: {difference_count}")


Number of differences: 26834


In [5]:
#lol_df = lol_df.drop(columns=['redFirstBlood', 'blueFirstBlood'])
lol_df = lol_df.drop_duplicates()

In [6]:
lol_df = pd.get_dummies(lol_df)
lol_df

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,...,"redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON', 'AIR_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",redDragnoType_['WATER_DRAGON'],redDragnoType_[]
0,4247263043,0,24081,1190,44,8.8,309,74,0,8,...,False,False,False,True,False,False,False,False,False,False
1,4247155821,1,24162,2212,46,9.2,393,64,0,5,...,False,False,False,False,False,False,False,False,False,True
2,4243963257,0,22413,1563,41,8.2,300,62,0,5,...,False,False,False,False,False,False,False,False,False,False
3,4241678498,0,23837,3197,46,9.2,370,96,0,6,...,False,False,False,False,False,False,False,False,False,True
4,4241538868,1,27688,3663,44,8.8,381,66,0,9,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26829,4143231833,0,26110,1535,46,9.2,289,73,0,15,...,False,False,False,False,False,False,False,False,True,False
26830,4157911901,0,57503,3293,62,12.4,329,28,0,27,...,False,False,False,False,False,False,False,False,False,True
26831,3764171638,0,26091,2986,47,9.4,338,86,0,7,...,False,False,False,False,False,False,False,False,False,True
26832,4110201724,1,24734,4289,45,9.0,328,64,0,11,...,False,False,False,False,False,False,False,False,False,False


# Dropping features which are highly correlated with each other

In [7]:
# Defining correlation Matrix
lol_corr = lol_df.corr()

Iterate through the correlation matrix

In [8]:
# Threshold to consider highly correlated features
high_threshold = 0.9

# Find list pairings of highly correlated features (excluding 'blueWins')
high_corr_pairs = []
for i in range(len(lol_corr.columns)):
    for j in range(i + 1, len(lol_corr.columns)):
        if abs(lol_corr.iloc[i, j]) > high_threshold and lol_corr.index[i] != 'blueWins' and lol_corr.columns[j] != 'blueWins':
            high_corr_pairs.append([lol_corr.index[i], lol_corr.columns[j]])

# Compare correlations with 'blueWins' and determine which features to drop
features_to_drop = []
for pair in high_corr_pairs:
    feature1, feature2 = pair
    if abs(lol_corr.loc[feature1, 'blueWins']) >= abs(lol_corr.loc[feature2, 'blueWins']):
        features_to_drop.append(feature2)  # Drop the feature with lower correlation with blueWins
    else:
        features_to_drop.append(feature1)

# Remove duplicates from the features_to_drop list
features_to_drop = list(set(features_to_drop))

In [9]:
# Dropping correlated features
lol_df_dropped = lol_df.drop(columns=features_to_drop)
lol_df_dropped = lol_df_dropped.drop(columns="gameId")

# Dropping features which are lowly correlated with target

Iterate through the correlation matrix

In [ ]:
# Dropping features with low correlation with target

# Defining new correlation Matrix
lol_corr = lol_df_dropped.corr()

# Threshold to consider low correlation features with 'blueWins'
low_threshold = 0.05

# Find features with correlation below the threshold with 'blueWins'
low_corr_features = []
for feature in lol_corr.index:
    if feature != 'blueWins' and abs(lol_corr.loc[feature, 'blueWins']) < low_threshold:
        low_corr_features.append(feature)

# Output the list of features with low correlation to 'blueWins'
low_corr_features

In [ ]:
lol_df_dropped_2 = lol_df_dropped.drop(columns=low_corr_features)

In [ ]:
lol_df_dropped_2

# KNN Machin Learning

In [19]:
features = lol_df_dropped_2.drop(columns=['blueWins'])
target = lol_df_dropped_2["blueWins"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

#Normalize Data after Train Split
normalizer = MinMaxScaler() #define normalizer

normalizer.fit(X_train)

X_train_norm = normalizer.transform(X_train) # Normalize 80% training dats
X_test_norm = normalizer.transform(X_test) # Normalize 20% Testing Data

#Apply to test and training data
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)


In [21]:
knn = KNeighborsRegressor(n_neighbors=10)

knn.fit(X_train_norm, y_train)

KNeighborsRegressor(n_neighbors=10)

In [22]:
knn.score(X_test_norm, y_test)

0.9919930205679044

In [23]:
knn.predict(X_test_norm)

array([0., 1., 1., ..., 0., 0., 1.])

# Model Tuning and GridSearch

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

model = RandomForestClassifier()

grid_search = GridSearchCV(estimator= model, param_grid=param_grid, scoring="accuracy")

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy')